In [1]:
import json
import xml.etree.ElementTree as ET
#neu
import requests
from pathlib import Path
raw_xml = """<?xml version="1.0" encoding="UTF-8"?>
<definitions xmlns="http://www.omg.org/spec/BPMN/20100524/MODEL"
             id="Definitions_1"
             targetNamespace="http://bpmn.io/schema/bpmn">
  <process id="Process_1" isExecutable="false">
    <startEvent id="StartEvent_1">
      <outgoing>Flow_1</outgoing>
    </startEvent>

    <task id="Task_1" name="Do something">
      <incoming>Flow_1</incoming>
      <outgoing>Flow_2</outgoing>
    </task>

    <endEvent id="EndEvent_1">
      <incoming>Flow_2</incoming>
    </endEvent>

    <sequenceFlow id="Flow_1" sourceRef="StartEvent_1" targetRef="Task_1"/>
    <sequenceFlow id="Flow_2" sourceRef="Task_1" targetRef="EndEvent_1"/>
  </process>
</definitions>
"""
print(raw_xml[:200])

r = requests.post("http://localhost:3001/process-bpmn", json={"bpmnXml": raw_xml}, timeout=60)
print("HTTP:", r.status_code)
r.raise_for_status()

layouted_xml = r.json()["layoutedXml"]

print("BPMNShape present:", "BPMNShape" in layouted_xml)
print("BPMNEdge present:", "BPMNEdge" in layouted_xml)

out = Path("data/output")
out.mkdir(parents=True, exist_ok=True)
(out / "process_layouted_test.bpmn").write_text(layouted_xml, encoding="utf-8")
print("Wrote:", out / "process_layouted_test.bpmn")
#endneu

#from bpmn_assistant.config import logger
#from bpmn_assistant.services import BpmnProcessTransformer

import logging


class BpmnXmlGenerator:
    """
    Class to generate BPMN XML from the BPMN process data in JSON format.
    """

    def __init__(self):
        self.transformer = BpmnProcessTransformer()

    def create_bpmn_xml(self, process: list[dict]) -> str:
        """
        Create BPMN XML from the process data.
        Args:
            process: BPMN process structure generated by the LLM.
            The BPMN XML string.
        """

        transformed_process = self.transformer.transform(process)
        logger.debug(
            f"Transformed process:\n{json.dumps(transformed_process, indent=2)}"
        )

        # Create the root element (definitions)
        root = ET.Element("definitions")
        root.set("xmlns", "http://www.omg.org/spec/BPMN/20100524/MODEL")
        root.set("xmlns:bpmndi", "http://www.omg.org/spec/BPMN/20100524/DI")
        root.set("xmlns:dc", "http://www.omg.org/spec/DD/20100524/DC")
        root.set("xmlns:di", "http://www.omg.org/spec/DD/20100524/DI")
        root.set("id", "definitions_1")

        # Create the process element
        process_element = ET.SubElement(root, "process")
        process_element.set("id", "Process_1")
        process_element.set("isExecutable", "false")

        # Add elements
        for element in transformed_process["elements"]:
            elem = ET.SubElement(process_element, element["type"])
            elem.set("id", element["id"])

            # Add label if it exists
            if element["label"]:
                elem.set("name", element["label"])

            # Add incoming and outgoing flows as child elements
            for incoming in element["incoming"]:
                ET.SubElement(elem, "incoming").text = incoming
            for outgoing in element["outgoing"]:
                ET.SubElement(elem, "outgoing").text = outgoing

        # Add flows
        for flow in transformed_process["flows"]:
            seq_flow = ET.SubElement(process_element, "sequenceFlow")
            seq_flow.set("id", flow["id"])
            seq_flow.set("sourceRef", flow["sourceRef"])
            seq_flow.set("targetRef", flow["targetRef"])

            # Add condition if it exists
            if flow["condition"]:
                seq_flow.set("name", flow["condition"])

        xml_string = ET.tostring(root, encoding="unicode")

        return xml_string

<?xml version="1.0" encoding="UTF-8"?>
<definitions xmlns="http://www.omg.org/spec/BPMN/20100524/MODEL"
             id="Definitions_1"
             targetNamespace="http://bpmn.io/schema/bpmn">
  <pr
HTTP: 200
BPMNShape present: True
BPMNEdge present: True
Wrote: data\output\process_layouted_test.bpmn
